# My Playground for Neural Network

In [902]:
from utils import *
from stratified import *
from run import *
from neuralnetwork import *

In [903]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [904]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [905]:
n_ohe_house_data = normalizetrain(ohe_house_data,ohe_house_category)[0]
n_ohe_wine_data = normalizetrain(ohe_wine_data,ohe_wine_category)[0]
n_ohe_cmc_data = normalizetrain(ohe_cmc_data,ohe_cmc_category)[0]
n_ohe_example_data = normalizetrain(ohe_example_data,ohe_example_category)[0]
n_ohe_cancer_data = normalizetrain(ohe_cancer_data,ohe_cancer_category)[0]

In [906]:
def g(x): # sigmoid function
    return np.exp(x) / (1 + np.exp(x))

In [907]:
def transposelistoflist(l):
    newlistoflist = []
    for i in range(len(l[0])):
        newlist = []
        for j in range(len(l)):
            newlist.append(l[j][i])
        newlistoflist.append(newlist)
    return newlistoflist

In [908]:
def gradientD(weights_list,deltalist,attributelist,biasterm=True):
    gradlist = []
    for i in range(len(weights_list)):
        attributenow = attributelist[i]
        deltanow = np.array([deltalist[i]]).T
        dotproduct = deltanow*attributenow
        # print('dotshape',dotproduct.shape)
        gradlist.append(dotproduct)
    return gradlist

In [909]:
def g(x): # sigmoid function
    return 1/(1 + np.exp(-x))

In [910]:
# Forward propagation vectorized
def neural_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 435, lambda_reg = 0.07, learning_rate = 0.0001):
    normed_ohe_copy = normed_hotted_data.copy()
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].T
        output_data = onebatch[outputindex].T
        # input_data_mean = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].T

        # forward propagation
        instance_index = 0
        j = 0
        listofgradient = []
        for one_instance in input_data:
            current_layer_a = np.append(1,one_instance) if b == 1 else one_instance
            # input layer is the current layer
            current_layer_index = 0
            output_real = output_data[instance_index]
            attributesnobias = [one_instance]
            attributeswbias = [current_layer_a]
            for theta in weights_list:
                z = np.dot(theta,current_layer_a)
                a = g(z)
                current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
                attributesnobias.append(a)
                attributeswbias.append(current_layer_a)
                current_layer_index += 1

            output_predict = current_layer_a # the last attribute is the output for this batch.
            instance_index += 1
            j += costfunction(output_real,output_predict)

            # calculate delta blame (back propagation)
            listofdelta = blame(output_real,output_predict,weights_list, attributeswbias)
            thisgradient = gradientD(weights_list,listofdelta,attributeswbias,biasterm)
            listofgradient.append(thisgradient)
        
        gradientP = [lambda_reg*t for t in weights_list]
        grad_D_transpose = transposelistoflist(listofgradient)
        grad_D_sum = [np.sum(t,axis=0) for t in grad_D_transpose]
        gradients_batch = []
        for i in range(len(grad_D_sum)):
            gradients_batch.append((grad_D_sum[i] + gradientP[i])*(1/instance_index))
        
        j /= (instance_index+1)
        s = sumofweights(weights_list,bias=b)*lambda_reg/(2*(instance_index+1))
        allj = j+s # total cose with regularization

        # update weights
        for i in range(len(weights_list)):
            weights_list[i] -= learning_rate*gradients_batch[i]

    return weights_list, allj, j

In [911]:
normalizeinuse = n_ohe_cancer_data
categoryinuse = ohe_cancer_category

In [912]:
layerdataparameter = [5,5,5,5,5,5] # [First layer, Second layer, Third layer], not including the bias and input & output layer
initweights = initialize_weights(categoryinuse,layerdataparameter)
kk,jj,pj = neural_network(normalizeinuse,categoryinuse,initweights)

In [913]:
j_previous = 99
epsilon = 80
weight, j_now, purej = neural_network(normalizeinuse,categoryinuse,initweights)
epsilon = j_previous - j_now
j_previous = j_now
smallestj = j_now
smallestpure = purej
count = 0
while epsilon > 0.00001 and j_now >= smallestj and purej >= smallestpure:
    print(count)
    count+=1
    weight, j_now, purej = neural_network(normalizeinuse,categoryinuse,weight)
    print('purej',purej)
    print('j now is',j_now)
    smallestj = min(smallestj,j_now)
    smallestpure = min(smallestpure,purej)
    epsilon = abs(j_previous - j_now)
    print(epsilon)
    j_previous = j_now

0
purej 0.48884628445299305
j now is 1.3663977796841509
0.5301987343191052
1
purej 0.9670246105054194
j now is 1.8377912446678963
0.47139346498374546
2
purej 0.5131148052752802
j now is 1.3774077972831018
0.46038344738479453
3
purej 0.9267247108844991
j now is 1.7850055159846154
0.4075977187015136
4
purej 0.5365177081713868
j now is 1.389011524808853
0.3959939911757624
5
purej 0.5473652932060711
j now is 1.3944374691397337
0.0054259443308806166
6
purej 0.558209627958166
j now is 1.4000808405045033
0.005643371364769667
7
purej 0.8573423573137708
j now is 1.694211711409863
0.2941308709053596
8
purej 0.8422186784647014
j now is 1.674292458964671
0.019919252445191882
9
purej 0.8270195261213389
j now is 1.6544517520284736
0.01984070693619744
10
purej 0.6008828326093186
j now is 1.4238539142351267
0.23059783779334686
11
purej 0.6108378438848696
j now is 1.4294890652627057
0.005635151027578944
12
purej 0.620791263491356
j now is 1.435245384772735
0.005756319510029195
13
purej 0.77536494775723

In [914]:
print(j_now)
correctweight = weight

0.5014158857287334


In [915]:
def traverse_network(input, weight):
    for i in range(len(weight)):
        input = np.append(1,input) if i != (len(weight)) else input
        z = np.dot(weight[i],input)
        a = g(z)
        input = a
    output = a
    n = 0
    for item in output:
        if item< 0.5:
            output[n] = 0
        else:
            output[n] = 1
        n += 1
    return output

In [916]:
def predictusingnetwork(data, category, weight):
    n = 0
    inputindex, outputindex = [],[]
    for i in category:
        if category[i] != 'class_numerical':
            inputindex.append(n)
        else:
            outputindex.append(n)
        n += 1
    predictoutput = traverse_network(data[inputindex],weight)
    realoutput = data[outputindex]
    # print(predictoutput, realoutput)
    return predictoutput, realoutput

In [917]:
allcount = 0
correctcount = 0
for instance in normalizeinuse:
    predict, real = predictusingnetwork(instance,categoryinuse,correctweight)
    for i in range(len(predict)):
        if predict[i] == real[i]:
            correctcount += 1
        allcount += 1
    
print(correctcount/allcount)

0.6552217453505007
